# Identifying even or odd number of digits in any whole number without using number of digits in features
In this notebook, I wanted to explore an ideal Neural Network for identifying if a passed number has even or odd number of digits based on the number alone.
- This would entail that the model needs to understand that $digits = log(|n|) + 1$, where $n$ is the number of digits would generally give us the answer for if it is even or odd. But how can a Neural Network do this, and more importantly choice of the NN?

In [13]:
import torch

In [14]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
    print(f"MPS device found: {mps_device}")
else:
    print("MPS device not found or not built with MPS enabled.")
    print("Check macOS version (12.3+) and PyTorch installation.")


tensor([1.], device='mps:0')
MPS device found: mps


In [15]:
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import tqdm


# Data Generation
We need to generate uniform data across all lengths evenly. Now if we generate even number of number for every length, that wouldn't make sense because for example we only have 10 numbers between 0 and 9, but we have 9000 number between 1000 and 9999. But how would we sample the different numbers within each range? 

- We need the number of data points to indcrease with increased number of digits. This can be increased randomly or in multiples of a certain number.
- We can choose to have data points for all ranges or not.

*Solution?*
- Let's test on 3 different dataset configurations: One with each setting mentioned in above bullet points.

In [16]:
def generate_dataset(n_samples:int=10e6, exp=15, random_ranges=False, all_ranges=True):
    ## If random ranges is True, that means we can sample by the exponents and not numbers itself
    if random_ranges:
        exponents = np.random.randint(0, exp, size=n_samples)
        
        unique_exps, counts = np.unique(exponents, return_counts=True)
        # unique_exps = [0, 1, 2]
        # counts = [2, 3, 4]  <- exp 0 appears 2 times, exp 1 appears 3 times, 
        # exp 2 appears 4 times

        
        numbers = []
        for e, count in zip(unique_exps, counts):
            lower, upper = 10**e, 10**(e+1)
            range_size = upper - lower

            if count <= range_size:
                nums = np.random.choice(range_size, size=count, replace=False)
                # choice means duplicate numbers are not allowed
                numbers.extend(nums)
            else:
                nums = np.random.randint(lower, upper, size=count)
                numbers.extend(np.unique(nums))
                
    ## otherwise, we do it proportional to the log size of random range from 
    ## -exp to +exp to see how important each order of magnitude is
    else:
        # we will use weight technique to weigh how many samples we want
        # from each order of magnitude

        # all_ranges means we want to sample from all ranges equally
        if all_ranges:
            weights = [9 * (10 ** i) for i in range(exp)] # so like 9, 90, 900, 9000, ...
            total_weights = sum(weights)
            
            probabilities = [n_samples * (w / total_weights) for w in weights]
        else:
            weights = np.random.randint(1, exp, size=exp)
            total_weights = sum(weights)
            probabilities = [n_samples * (w / total_weights) for w in weights]
        numbers = []
        for i, p in enumerate(probabilities):
            count = int(p)

            if count == 0:
                continue

            lower, upper = 10**i, 10**(i+1)
            range_size = upper - lower
            if count <= range_size:
                nums = np.random.choice(np.arange(lower, upper), size=count, replace=False)
                numbers.extend(nums)
            else:
                nums = np.random.randint(lower, upper, size=count)
                numbers.extend(np.unique(nums))


    labels = [1 if len(str(n)) % 2 == 0 else 0 for n in numbers]
    return np.array(numbers), np.array(labels)

In [17]:
# Data Type 1: Balanced dataset from all ranges
X, Y = generate_dataset(n_samples=int(10e5), exp=6)
print(X.shape, Y.shape)
print(X[59990:60000], Y[59990:60000])

(999999,) (999999,)
[20390 25542 38378 48203 45026 11157 93389 74905 27555 59441] [0 0 0 0 0 0 0 0 0 0]


### Data splits

In [18]:
# data splits
split = {"train": 0.7, "val": 0.2, "test": 0.1}
n_samples = X.shape[0]

# split randomly
indices = np.random.permutation(n_samples)

X_train = X[indices[:int(split["train"] * n_samples)]]
X_val = X[indices[int(split["train"] * n_samples):int((split["train"] + split["val"]) * n_samples)]]
X_test = X[indices[int((split["train"] + split["val"]) * n_samples):]]

Y_train = Y[indices[:int(split["train"] * n_samples)]]
Y_val = Y[indices[int(split["train"] * n_samples):int((split["train"] + split["val"]) * n_samples)]]
Y_test = Y[indices[int((split["train"] + split["val"]) * n_samples):]]

# Neural Networks
As mentioned at the start, we need a NN that can learn log functions. For this purpose, simple MLPs might be more difficult to work with. Given below is a list of networks we would test and compare for our problem.

### 1. Gradient Boosted Trees
- Ensemble models can be a good choice for problems like this where models can learn from data splits. 
- In the case of GB trees, splits will be recurring to avoid data bias which may occur based on seen data. 
- If we take our own dataset, certain ranges have much lower number of samples (ex. 0-9) in comparison to others (ex. 1000 - 9999).
- We can hypothesize that GB trees can learn a boundary function on repetitive training on splits.

### 2. Random Forests
- While GB trees could learn sequentially for handling any data bias, random forests can deal with variance in the data.
- While we can hypothesize that GB Trees would perform better than RF, this could still be a good contender.
- RF has the potential for learning the power-of-10 boundaries where the label changes from 0 to 1 and back to 0.

### 3. MLP with RBF Kernel
- While we can hypothesize that this will not perform as well as the latter 2, RBF kernels can be good at drawing decision non-linear decision boundaries.

## Gradient Boosted Trees

In [19]:
# We will use XGBoost for this task
!pip3 install xgboost
import xgboost as xgb

In [20]:
# xgboost requires our data to be in 2D array
DX_train = X_train.reshape(-1, 1)
DX_val = X_val.reshape(-1, 1)
DX_test = X_test.reshape(-1, 1)

In [21]:
gbt_model = xgb.XGBClassifier(
    max_depth=12,
    n_estimators=100,
    learning_rate=0.1,
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42
)

print("Training Gradient Boosted Trees Model...")
gbt_model.fit(DX_train, Y_train)

Training Gradient Boosted Trees Model...


,"objective objective: str | xgboost.sklearn._SklObjWProto | typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]] | NoneSpecify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: float | typing.List[float] | NoneThe initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.List[xgboost.callback.TrainingCallback] | NoneList of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: float | NoneSubsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: float | NoneSubsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: float | NoneSubsample ratio of columns when constructing each tree.,None
,"device device: str | None.. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: int | None.. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: str | typing.List[str | typing.Callable] | typing.Callable | None.. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes from sklearn.metrics import mean_absolute_error X, y = load_diabetes(return_X_y=True) reg = xgb.XGBRegressor( tree_meth

In [22]:
y_pred = gbt_model.predict(DX_test)
accuracy = np.mean(y_pred == Y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 99.62%


In [23]:
# lets see where it fails
[(len(str(DX_test[i])), Y_test[i], y_pred[i]) for i in range(len(Y_test)) if Y_test[i] != y_pred[i]][:10]

[(8, np.int64(1), np.int64(0)),
 (8, np.int64(1), np.int64(0)),
 (7, np.int64(0), np.int64(1)),
 (7, np.int64(0), np.int64(1)),
 (7, np.int64(0), np.int64(1)),
 (7, np.int64(0), np.int64(1)),
 (7, np.int64(0), np.int64(1)),
 (5, np.int64(0), np.int64(1)),
 (7, np.int64(0), np.int64(1)),
 (5, np.int64(0), np.int64(1))]

## Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=12,
    min_samples_split=5,
    min_samples_leaf=2,
    bootstrap=True,
    random_state=42
)

print("Training Random Forest Model...")
rf_model.fit(DX_train, Y_train)

Training Random Forest Model...


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",12
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",5
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",2
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric(y

In [25]:
y_pred = rf_model.predict(DX_val)
accuracy = np.mean(y_pred == Y_val)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Validation Accuracy: 100.00%


In [26]:
y_pred = rf_model.predict(DX_test)

accuracy = np.mean(y_pred == Y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 100.00%


## Simple MLP with RBF Kernel

In [27]:
'''
In RBF kernel, we will want the decision boundary to look like concentric circles
around the origin for each reference point - reference point here being the 
boundaries from one order of magnitude to another.
'''

def rbf_features(X, centers, gamma=1.0):
    '''
    X: numpy array of input numbers (n_samples, 1)
    centers: reference points (powers of 10)
    gamma: parameter for RBF kernel for width (higher gamma = narrower kernel)
    '''
    # || X - center ||^2
    distances_sq = (X - centers.reshape(1, -1)) ** 2

    # rbf kernel = exp(-gamma * distance^2)
    rbf = np.exp(-gamma * distances_sq)
    return rbf

In [28]:
positive_centers = np.array([10**i for i in range(1, 6)])  # 10, 100, ..., 10^15
negative_centers = np.array([-10**i for i in range(1, 6)])  # -10, -100, ..., -10^15
all_centers = np.concatenate([positive_centers, negative_centers])

# we choose a smaller gamma to have wider kernels since our span of data is large
gamma = 1e-10

PX_train = rbf_features(DX_train, all_centers, gamma=gamma)
PX_val = rbf_features(DX_val, all_centers, gamma=gamma)
PX_test = rbf_features(DX_test, all_centers, gamma=gamma)


In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
# we want to standaridize the rbf features
scaler = StandardScaler()
PX_train = scaler.fit_transform(PX_train)
PX_val = scaler.transform(PX_val)

#MLP model
mlp_model = MLPClassifier(
    hidden_layer_sizes = (128, 64, 32), # 3 hidden layers
    activation='relu',
    solver='adam',
    max_iter=100,
    learning_rate_init=0.001,
    random_state=42,
    batch_size=256,
    early_stopping=True,
    verbose=True
)

#Train
print("Training MLP Model with RBF Features...")
mlp_model.fit(PX_train, Y_train)

Training MLP Model with RBF Features...
Iteration 1, loss = 0.02144518
Validation score: 0.998114
Iteration 2, loss = 0.00459264
Validation score: 0.997857
Iteration 3, loss = 0.00383533
Validation score: 0.998229
Iteration 4, loss = 0.00368993
Validation score: 0.997514
Iteration 5, loss = 0.00363392
Validation score: 0.998357
Iteration 6, loss = 0.00325532
Validation score: 0.997786
Iteration 7, loss = 0.00344982
Validation score: 0.999500
Iteration 8, loss = 0.00299353
Validation score: 0.999643
Iteration 9, loss = 0.00309790
Validation score: 0.998800
Iteration 10, loss = 0.00317896
Validation score: 0.998100
Iteration 11, loss = 0.00301697
Validation score: 0.999400
Iteration 12, loss = 0.00293374
Validation score: 0.998729
Iteration 13, loss = 0.00299932
Validation score: 0.999329
Iteration 14, loss = 0.00271642
Validation score: 0.999243
Iteration 15, loss = 0.00285073
Validation score: 0.998886
Iteration 16, loss = 0.00273024
Validation score: 0.999529
Iteration 17, loss = 0.00

,"hidden_layer_sizes hidden_layer_sizes: array-like of shape(n_layers - 2,), default=(100,)The ith element represents the number of neurons in the ithhidden layer.","(128, ...)"
,"activation activation: {'identity', 'logistic', 'tanh', 'relu'}, default='relu'Activation function for the hidden layer.- 'identity', no-op activation, useful to implement linear bottleneck, returns f(x) = x- 'logistic', the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).- 'tanh', the hyperbolic tan function, returns f(x) = tanh(x).- 'relu', the rectified linear unit function, returns f(x) = max(0, x)",'relu'
,"solver solver: {'lbfgs', 'sgd', 'adam'}, default='adam'The solver for weight optimization.- 'lbfgs' is an optimizer in the family of quasi-Newton methods.- 'sgd' refers to stochastic gradient descent.- 'adam' refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy BaFor a comparison between Adam optimizer and SGD, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_training_curves.py`.Note: The default solver 'adam' works pretty well on relativelylarge datasets (with thousands of training samples or more) in terms ofboth training time and validation score.For small datasets, however, 'lbfgs' can converge faster and performbetter.",'adam'
,"alpha alpha: float, default=0.0001Strength of the L2 regularization term. The L2 regularization termis divided by the sample size when added to the loss.For an example usage and visualization of varying regularization, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_alpha.py`.",0.0001
,"batch_size batch_size: int, default='auto'Size of minibatches for stochastic optimizers.If the solver is 'lbfgs', the classifier will not use minibatch.When set to ""auto"", `batch_size=min(200, n_samples)`.",256
,"learning_rate learning_rate: {'constant', 'invscaling', 'adaptive'}, default='constant'Learning rate schedule for weight updates.- 'constant' is a constant learning rate given by 'learning_rate_init'.- 'invscaling' gradually decreases the learning rate at each time step 't' using an inverse scaling exponent of 'power_t'. effective_learning_rate = learning_rate_init / pow(t, power_t)- 'adaptive' keeps the learning rate constant to 'learning_rate_init' as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol if 'early_stopping' is on, the current learning rate is divided by 5.Only used when ``solver='sgd'``.",'constant'
,"learning_rate_init learning_rate_init: float, default=0.001The initial learning rate used. It controls the step-sizein updating the weights. Only used when solver='sgd' or 'adam'.",0.001
,"power_t power_t: float, default=0.5The exponent for inverse scaling learning rate.It is used in updating effective learning rate when the learning_rateis set to 'invscaling'. Only used when solver='sgd'.",0.5
,"max_iter max_iter: int, default=200Maximum number of iterations. The solver iterates until convergence(determined by 'tol') or this number of iterations. For stochasticsolvers ('sgd', 'adam'), note that this determines the number of epochs(how many times each data point will be used), not the number ofgradient steps.",100
,"shuffle shuffle: bool, default=TrueWhether to shuffle samples in each iteration. Only used whensolver='sgd' or 'adam'.",True
,"random_state random_state: int, RandomState instance, default=NoneDetermines random number generation for weights and biasinitialization, train-test split if early stopping is used, and batchsampling when solver='sgd' or 'adam'.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",42


In [30]:
y_pred = mlp_model.predict(PX_val)
accuracy = np.mean(y_pred == Y_val)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Validation Accuracy: 99.99%


In [31]:
y_pred = mlp_model.predict(PX_test)
accuracy = np.mean(y_pred == Y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 96.08%


# Testing
Now its important to see how each model performs with data outside the training range. Training rangeonly covers the top 

In [ ]:
low_range, high_range = 6, 15
size = 10e3

X_new, Y_new = generate_dataset(n_samples=int(size), exp=high_range, random_ranges=True, all_ranges=False)
DX_new = X_new.reshape(-1, 1)
PX_new = rbf_features(DX_new, all_centers, gamma=gamma)
PX_new = scaler.transform(PX_new)